In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from deep_sort_realtime.deepsort_tracker import DeepSort
from sorted_alpha import sorted_alpha
import matplotlib.pyplot as plt

In [ ]:
tracker = DeepSort(max_age=1000, override_track_class=None, embedder_gpu=False)


In [ ]:
def find_min_round(array):
    array = np.array(array)#remove the arms from the equation cause the bounding boxes are too large 
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    array = np.array(array)
    x = round(np.max(array))
    return(x)

In [ ]:
detection = Detection()

In [ ]:
pose = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1047_play'
detections = sorted_alpha(pose)

In [ ]:
skeletons = []
for det in detections: 
    temp_list = []
    df = pd.read_csv(os.path.join(pose,det))
    columns = df.columns[1:]
    probs = [df.iloc[:,column+1].mean() for column in range(2, len(columns), 3)]
    detections_x = [df.iloc[:,column+1].min() for column in range(0, len(columns), 3)]                                   
    detections_y  = [df.iloc[:,column+1].min() for column in range(1, len(columns), 3)]
    width = [df.iloc[:,column+1].max() for column in range(0, len(columns), 3)]
    height = [df.iloc[:,column+1].max() for column in range(1, len(columns), 3)]
    k=1
    for x,y,w,h,p in zip(detections_x, detections_y, width, height, probs):
        temp_list.append(([x,y,w-x,h-y], round(p, 2), 'person '+str(k)))
        k+=1
    skeletons.append(temp_list)
             

In [ ]:
skeletons[100]


In [ ]:
a = [5, 7, 11, 4, 5]

for previous, current in zip(a, a[1:]):
    print([previous, current])

In [ ]:
video = '/Users/andrei-macpro/Documents/Data/videos/play_videos/1047_play.mp4'
cap = cv2.VideoCapture(video)

In [ ]:
(np.ones((1080, 1920, 3), dtype=np.uint8) * 255).shape

In [ ]:
tracker.delete_all_tracks()

In [ ]:
cap.set(1,1)
ret, frame_1 = cap.read()
cap.set(1,2)
ret, frame_2 = cap.read()
cap.set(1,3)
ret, frame_3 = cap.read()

In [ ]:
tracks=[]

In [ ]:
tracks = tracker.update_tracks(skeletons[0], frame=frame_1)

In [ ]:
for track in tracks:
        print(track.track_id)
        print(track.to_tlwh())
        

In [ ]:
tracker.generate_embeds(frame=frame_1, raw_dets = skeletons[0])

In [ ]:
embeddings = []
for det in range(len(skeletons)):
    cap.set(1,det+1)
    ret, frame = cap.read()
    if ret==False:
        break
    embeddings.append(tracker.generate_embeds(frame=frame, raw_dets = skeletons[det]))

    
    

In [ ]:
embeddings[0]

In [ ]:
len(embeddings), len(skeletons)

In [ ]:
len(embeddings[100][0]), len(embeddings[300][1])

In [ ]:
skeletons[1]

In [ ]:
bbs = tracker.crop_bb(frame_2, skeletons[1])

In [ ]:
plt.imshow(bbs[1])

In [ ]:
cap.set(1,5000)
ret, frame5000 = cap.read()

In [ ]:
plt.imshow(frame_1)

In [ ]:
skeletons[1]

In [ ]:
raw_dets = skeletons[1]
frame = frame_2
crops = []
im_height, im_width = frame.shape[:2]
for detection in raw_dets:
    l, t, w, h = [int(x) for x in detection[0]]
    r = l + w
    b = t + h
    crop_l = max(0, l)
    crop_r = min(im_width, r)
    crop_t = max(0, t)
    crop_b = min(im_height, b)
    crops.append(frame[crop_t:crop_b, crop_l:crop_r])

In [ ]:
for detection in raw_dets:
    l, t, w, h = [int(x) for x in detection[0]]
    print(l+w)
    print(min(im_width, l+w))
    

In [ ]:
max(0,384)

In [ ]:
im_width